In [1]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import kagglehub as kh
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras import layers, models
from keras.applications import ResNet50
from keras.applications.resnet50 import preprocess_input

In [2]:
path = kh.dataset_download("afridirahman/brain-stroke-ct-image-dataset")

print("Path to dataset files:", path)


Using Colab cache for faster access to the 'brain-stroke-ct-image-dataset' dataset.
Path to dataset files: /kaggle/input/brain-stroke-ct-image-dataset


In [3]:
base_path =os.path.join(path, 'Brain_Data_Organised')
print(os.listdir(base_path))

['Stroke', 'Normal']


In [4]:
data_dir = base_path

In [5]:
img_size = (224, 224)    # ResNet50 expected input
batch_size = 16          # reduce if you have limited GPU memory
seed = 42
epochs = 10

In [6]:
# 2) Data generators (train / val / test split)
# We will use ImageDataGenerator with validation_split for a quick split.

train_datagen = ImageDataGenerator(
    preprocessing_function = preprocess_input,
    rescale=None,
    rotation_range=10,
    validation_split=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.05,
    height_shift_range=0.05,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)


In [7]:
train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='training',
    shuffle=True,
    seed = seed,
    color_mode='rgb'
)

val_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='validation',
    shuffle=True,
    seed = seed,
    color_mode='rgb'
)

# we'll use the validation set as 'val' and later show evaluation on it.

Found 2001 images belonging to 2 classes.
Found 500 images belonging to 2 classes.


In [8]:
print("Class indices:", train_generator.class_indices)

Class indices: {'Normal': 0, 'Stroke': 1}


In [9]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224,224, 3))
base_model.trainable = False  # freeze base

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


In [10]:
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.3),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(1, activation='sigmoid')
])

In [11]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       262,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,850,113 (90.98 MB)

 Trainable params: 262,401 (1.00 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [12]:
callbacks = [
    tf.keras.callbacks.ModelCheckpoint("best_brain_stroke_resnet.h5", save_best_only=True, monitor='val_loss'),
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True)
]

history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=epochs,
    callbacks=callbacks
)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 347ms/step - accuracy: 0.5535 - loss: 0.8295

126/126 ━━━━━━━━━━━━━━━━━━━━ 75s 491ms/step - accuracy: 0.5538 - loss: 0.8287 - val_accuracy: 0.6120 - val_loss: 0.6510
Epoch 2/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 37s 290ms/step - accuracy: 0.6473 - loss: 0.6442 - val_accuracy: 0.6220 - val_loss: 0.7043
Epoch 3/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 36s 284ms/step - accuracy: 0.6715 - loss: 0.6059 - val_accuracy: 0.6560 - val_loss: 0.6579
Epoch 4/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 36s 282ms/step - accuracy: 0.6694 - loss: 0.6023 - val_accuracy: 0.6300 - val_loss: 0.6574
Epoch 5/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 222ms/step - accuracy: 0.6873 - loss: 0.5799

126/126 ━━━━━━━━━━━━━━━━━━━━ 36s 285ms/step - accuracy: 0.6871 - loss: 0.5800 - val_accuracy: 0.6500 - val_loss: 0.6237
Epoch 6/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 39s 308ms/step - accuracy: 0.7028 - loss: 0.5697 - val_accuracy: 0.6420 - val_loss: 0.6540
Epoch 7/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 35s 282ms/step - accuracy: 0.7113 - loss: 0.5617 - val_accuracy: 0.6460 - val_loss: 0.7544
Epoch 8/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 35s 280ms/step - accuracy: 0.7165 - loss: 0.5310 - val_accuracy: 0.6360 - val_loss: 0.6894
Epoch 9/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 36s 282ms/step - accuracy: 0.6923 - loss: 0.5754 - val_accuracy: 0.6600 - val_loss: 0.6528


In [13]:
# Evaluation on validation set (acts as test if you didn't set aside test)
